# Indirect methods exercises

:::{exercise}
:label: indirect-methods-ex-jacobi

Using a pen and calculator and starting values of $x_i^{(0)} = 0$, calculate the first 2 iterations of the Jacobi method for solving the system of linear equations below and calculate the maximum value of the residual.

$$ \begin{align*}
    5x_1 +x_2 -x_3 +x_4 &=14,\\
    x_1 +4x_2 -x_3 -x_4 &=10,\\
    -x_1 -x_2 +5x_3 +x_4 &=-15,\\
    x_1 -x_2 +x_3 +3x_4 &=3.
\end{align*} $$
:::


In [173]:
import numpy as np
import subprocess 

def table_row(string, k, x, r):
    string += f"| {k} |"
    for i in range(len(x)):
        string += f"{x[i]:9.6f} | "
    string += f"{max(abs(r)):0.2e} |\n"

    return string

def jacobi_table(A, b, tol=1e-4):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    r = b - np.dot(A, x)
    
    string =  f"| $k$ |"
    for i in range(n):
        string += f" $x_{i+1}$ | "
    string += " max residual |\n"
    string += "|:--:|"
    for i in range(n+1):
        string += ":--:|"
    string += "\n"

    string  = table_row(string, 0, x, r)

    for k in range(1, maxiter):
        xold = np.copy(x)
        for i in range(n):
            sum_ = 0
            for j in range(n):
                if i != j:
                    sum_ += A[i,j] * xold[j]
        
            x[i] = (b[i] - sum_) / A[i,i]
            
        r = b - np.dot(A, x)

        if k < 10:
            string  = table_row(string, k, x, r)

        if max(abs(r)) < tol:
            break
    

    string += "| $\\vdots$ |"
    for i in range(n + 1):
        string += " $\\vdots$ | "
    string += " \n"

    string  = table_row(string, k, x, r)

    return string


def print_number(x):
    if x.dtype == np.int32:
        return f"{x}"
    elif abs(x) < 1e-10:
        return "0"
    else:
        return f"{x:0.6}"


def matrix_latex(A):
    m = A.shape[0]
    try:
        n = A.shape[1]
    except:
        n = 1

    string = "\\begin{pmatrix} "
    for i in range(m):
        if n == 1:
            string += print_number(A[i])
            if i < m - 1:
                string += " \\\ "
        
        else:
            for j in range(n):
                string += print_number(A[i,j])
                if j < n - 1:
                    string += " & "
                elif i < m - 1:
                    string += " \\\ "

    string += " \end{pmatrix}"

    return string


def jacobi_latex(A, b, tol=1e-4):
    n = len(b)
    x = np.zeros(n)
    ordinal = ["first", "second", "third"]
    string =  "The Jacobi method for this system is\n\n"
    string += "$$ \\begin{align*}\n"
    for i in range(n):
        string += f"    x_{{{i+1}}}^{{(k+1)}} &= \\frac{{1}}{{{A[i,i]}}} \\left( {b[i]}"
        for j in range(n):
            if i == j:
                continue
            if A[i,j] == 1:
                string += f" - x_{{{j+1}}}"
            elif A[i,j] == -1:
                string += f" + x_{{{j+1}}}"
            elif A[i,j] < 0:
                string += f" + {-A[i,j]} x_{{{j+1}}}"
            elif A[i,j] > 0:
                string += f" - {A[i,j]} x_{{{j+1}}}"
            if A[i,j] != 0:
                string += f"^{{(k)}}"
        
        if i == n - 1:
            string += " \\right). \n"
        else:
            string += " \\right), \\\ \n"
            
    string += "\\end{align*} $$ \n\n"
    string += "Using starting values of $\\mathbf{x} = \\mathbf{0}$. "
        
    for k in range(2):
        xo = np.copy(x)
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * xo[j]
        
            x[i] = s / A[i,i]
            
        r = b - np.dot(A, x)   
        
        string += f"Calculating the {ordinal[k]} iteration\n\n"
        string += "$$ \\begin{align*}\n"
        for i in range(n):
            string += rf"    x_{{{i+1}}}^{{({k+1})}} &= \frac{{1}}{{{A[i,i]}}} \left( {b[i]}"
            for j in range(n):
                if i == j:
                    continue
                if A[i,j] == 1:
                    if xo[j] >= 0:
                        string += " - " + print_number(xo[j])
                    else:
                        string += " + (" + print_number(xo[j]) + ")"

                elif A[i,j] == -1:
                    if xo[j] >= 0:
                        string += " + " + print_number(xo[j])
                    else:
                        string += " + (" + print_number(xo[j]) + ")"

                elif A[i,j] < 0:
                    string += f" - {-A[i,j]}(" + print_number(xo[j]) + ")"
                else:
                    string += f" - {A[i,j]}(" + print_number(xo[j]) + ")"

            string += " \\right) = " + print_number(x[i])
            if i == n - 1:
                string += ". \n"
            else:
                string += ", \\\ \n"
        
        string += "\\end{align*} $$\n\nCalculate the residual\n\n"
        string += "$$ \\begin{align*} \n"
        string += f"    \\mathbf{{r}}^{{({k+1})}} = \\mathbf{{b}} - A \\mathbf{{x}}^{{(1)}} = \n"
        string += "    " + matrix_latex(b) + " - \n    " + matrix_latex(A) + "\n    " + matrix_latex(x)+ " = \n    " + matrix_latex(r) + ".\n"
        string += "\\end{align*} $$\n\n"
        string += f"Since $\\max(| \\mathbf{{r}}^{{({k+1})}} |) = {max(abs(r)):0.6} > 10^{{-4}}$ we continue iterating. "
    
    return string
        
# Define linear system
A = np.array([[5, 1, -1, 1], [1, 4, -1, -1], [-1, -1, 5, 1], [1, -1, 1, 3]])
b = np.array([14, 10, -15, 3])

# Solve linear system
string = ":::{solution} indirect-methods-ex-jacobi\n:class: dropdown\n\n"
string += jacobi_latex(A, b)

string += "\n:::"

print(string)
# subprocess.run("pbcopy", text=True, input=string)
# subprocess.run("clip", text=True, input=string)

:::{solution} indirect-methods-ex-jacobi
:class: dropdown

The Jacobi method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= \frac{1}{5} \left( 14 - x_{2}^{(k)} + x_{3}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{2}^{(k+1)} &= \frac{1}{4} \left( 10 - x_{1}^{(k)} + x_{3}^{(k)} + x_{4}^{(k)} \right), \\ 
    x_{3}^{(k+1)} &= \frac{1}{5} \left( -15 + x_{1}^{(k)} + x_{2}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{4}^{(k+1)} &= \frac{1}{3} \left( 3 - x_{1}^{(k)} + x_{2}^{(k)} - x_{3}^{(k)} \right). 
\end{align*} $$ 

Using starting values of $\mathbf{x} = \mathbf{0}$. Calculating the first iteration

$$ \begin{align*}
    x_{1}^{(1)} &= \frac{1}{5} \left( 14 - 0 + 0 - 0 \right) = 2.8, \\ 
    x_{2}^{(1)} &= \frac{1}{4} \left( 10 - 0 + 0 + 0 \right) = 2.5, \\ 
    x_{3}^{(1)} &= \frac{1}{5} \left( -15 + 0 + 0 - 0 \right) = -3.0, \\ 
    x_{4}^{(1)} &= \frac{1}{3} \left( 3 - 0 + 0 - 0 \right) = 1.0. 
\end{align*} $$

Calculate the residual

$$ \begin{align*} 
    \mathbf{r}^{(1)} = \mathb

CompletedProcess(args='clip', returncode=0)

:::{solution} indirect-methods-ex-jacobi
:class: dropdown

The Jacobi method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= \frac{1}{5} \left( 14 - x_{2}^{(k)} + x_{3}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{2}^{(k+1)} &= \frac{1}{4} \left( 10 - x_{1}^{(k)} + x_{3}^{(k)} + x_{4}^{(k)} \right), \\ 
    x_{3}^{(k+1)} &= \frac{1}{5} \left( -15 + x_{1}^{(k)} + x_{2}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{4}^{(k+1)} &= \frac{1}{3} \left( 3 - x_{1}^{(k)} + x_{2}^{(k)} - x_{3}^{(k)} \right). 
\end{align*} $$ 

Using starting values of $\mathbf{x} = \mathbf{0}$. Calculating the first iteration

$$ \begin{align*}
    x_{1}^{(1)} &= \frac{1}{5} \left( 14 - 0 + 0 - 0 \right) = 2.8, \\ 
    x_{2}^{(1)} &= \frac{1}{4} \left( 10 - 0 + 0 + 0 \right) = 2.5, \\ 
    x_{3}^{(1)} &= \frac{1}{5} \left( -15 + 0 + 0 - 0 \right) = -3.0, \\ 
    x_{4}^{(1)} &= \frac{1}{3} \left( 3 - 0 + 0 - 0 \right) = 1.0. 
\end{align*} $$

Calculate the residual

$$ \begin{align*} 
    \mathbf{r}^{(1)} = \mathbf{b} - A \mathbf{x}^{(1)} = 
    \begin{pmatrix} 14 \\ 10 \\ -15 \\ 3 \end{pmatrix} - 
    \begin{pmatrix} 5 & 1 & -1 & 1 \\ 1 & 4 & -1 & -1 \\ -1 & -1 & 5 & 1 \\ 1 & -1 & 1 & 3 \end{pmatrix}
    \begin{pmatrix} 2.8 \\ 2.5 \\ -3.0 \\ 1.0 \end{pmatrix} = 
    \begin{pmatrix} -6.5 \\ -4.8 \\ 4.3 \\ 2.7 \end{pmatrix}.
\end{align*} $$

Since $\max(| \mathbf{r}^{(1)} |) = 6.5 > 10^{-4}$ we continue iterating. Calculating the second iteration

$$ \begin{align*}
    x_{1}^{(2)} &= \frac{1}{5} \left( 14 - 2.5 + (-3.0) - 1.0 \right) = 1.5, \\ 
    x_{2}^{(2)} &= \frac{1}{4} \left( 10 - 2.8 + (-3.0) + 1.0 \right) = 1.3, \\ 
    x_{3}^{(2)} &= \frac{1}{5} \left( -15 + 2.8 + 2.5 - 1.0 \right) = -2.14, \\ 
    x_{4}^{(2)} &= \frac{1}{3} \left( 3 - 2.8 + 2.5 + (-3.0) \right) = 1.9. 
\end{align*} $$

Calculate the residual

$$ \begin{align*} 
    \mathbf{r}^{(2)} = \mathbf{b} - A \mathbf{x}^{(1)} = 
    \begin{pmatrix} 14 \\ 10 \\ -15 \\ 3 \end{pmatrix} - 
    \begin{pmatrix} 5 & 1 & -1 & 1 \\ 1 & 4 & -1 & -1 \\ -1 & -1 & 5 & 1 \\ 1 & -1 & 1 & 3 \end{pmatrix}
    \begin{pmatrix} 1.5 \\ 1.3 \\ -2.14 \\ 1.9 \end{pmatrix} = 
    \begin{pmatrix} 1.16 \\ 3.06 \\ -3.4 \\ -0.76 \end{pmatrix}.
\end{align*} $$

Since $\max(| \mathbf{r}^{(2)} |) = 3.4 > 10^{-4}$ we continue iterating. 
:::

:::{exercise} 
:label: indirect-methods-ex-gauss-seidel

Repeat {ref}`indirect-methods-ex-jacobi` using the Gauss-Seidel method.
:::

In [186]:
def gauss_seidel_latex(A, b, tol=1e-4):
    n = len(b)
    x = np.zeros(n)
    ordinal = ["first", "second", "third"]
    string =  "The Gauss-Seidel method for this system is\n\n"
    string += "$$ \\begin{align*}\n"
    for i in range(n):
        string += f"    x_{{{i+1}}}^{{(k+1)}} &= \\frac{{1}}{{{A[i,i]}}} \\left( {b[i]}"
        for j in range(n):
            if i == j:
                continue
            if A[i,j] == 1:
                string += f" - x_{{{j+1}}}"
            elif A[i,j] == -1:
                string += f" + x_{{{j+1}}}"
            elif A[i,j] < 0:
                string += f" + {-A[i,j]} x_{{{j+1}}}"
            elif A[i,j] > 0:
                string += f" - {A[i,j]} x_{{{j+1}}}"
            if A[i,j] != 0:
                if j < i:
                    string += f"^{{(k+1)}}"
                else:
                    string += f"^{{(k)}}"
        
        if i == n - 1:
            string += " \\right). \n"
        else:
            string += " \\right), \\\ \n"
            
    string += "\\end{align*} $$ \n\n"
    string += "Using starting values of $\\mathbf{x} = \\mathbf{0}$. "
        
    for k in range(2):
        string += f"Calculating the {ordinal[k]} iteration\n\n"
        string += "$$ \\begin{align*}\n"

        for i in range(n):

            string += rf"    x_{{{i+1}}}^{{({k+1})}} &= \frac{{1}}{{{A[i,i]}}} \left( {b[i]}"
            sum_ = b[i]
            for j in range(n):

                if i != j:
                    sum_ -= A[i,j] * x[j]
        
                if i == j:
                    continue

                if A[i,j] == 1:
                    if x[j] >= 0:
                        string += " - " + print_number(x[j])
                    else:
                        string += " + (" + print_number(x[j]) + ")"

                elif A[i,j] == -1:
                    if x[j] >= 0:
                        string += " + " + print_number(x[j])
                    else:
                        string += " + (" + print_number(x[j]) + ")"

                elif A[i,j] < 0:
                    string += f" - {-A[i,j]}(" + print_number(x[j]) + ")"
                else:
                    string += f" - {A[i,j]}(" + print_number(x[j]) + ")"

            x[i] = sum_ / A[i,i]

            string += " \\right) = " + print_number(x[i])
            if i == n - 1:
                string += ". \n"
            else:
                string += ", \\\ \n"
            
        r = b - np.dot(A, x)   
        
        string += "\\end{align*} $$\n\nCalculate the residual\n\n"
        string += "$$ \\begin{align*} \n"
        string += f"    \\mathbf{{r}}^{{({k+1})}} = \\mathbf{{b}} - A \\mathbf{{x}}^{{(1)}} = \n"
        string += "    " + matrix_latex(b) + " - \n    " + matrix_latex(A) + "\n    " + matrix_latex(x)+ " = \n    " + matrix_latex(r) + ".\n"
        string += "\\end{align*} $$\n\n"
        string += f"Since $\\max(| \\mathbf{{r}}^{{({k+1})}} |) = {max(abs(r)):0.6} > 10^{{-4}}$ we continue iterating. "
    
    return string
        
# Define linear system
A = np.array([[5, 1, -1, 1], [1, 4, -1, -1], [-1, -1, 5, 1], [1, -1, 1, 3]])
b = np.array([14, 10, -15, 3])

# Solve linear system
string = ":::{solution} indirect-methods-ex-gauss-seidel\n:class: dropdown\n\n"
string += gauss_seidel_latex(A, b)

string += "\n:::"

print(string)
# subprocess.run("pbcopy", text=True, input=string)
# subprocess.run("clip", text=True, input=string)

:::{solution} indirect-methods-ex-gauss-seidel
:class: dropdown

The Gauss-Seidel method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= \frac{1}{5} \left( 14 - x_{2}^{(k)} + x_{3}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{2}^{(k+1)} &= \frac{1}{4} \left( 10 - x_{1}^{(k+1)} + x_{3}^{(k)} + x_{4}^{(k)} \right), \\ 
    x_{3}^{(k+1)} &= \frac{1}{5} \left( -15 + x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{4}^{(k)} \right), \\ 
    x_{4}^{(k+1)} &= \frac{1}{3} \left( 3 - x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{3}^{(k+1)} \right). 
\end{align*} $$ 

Using starting values of $\mathbf{x} = \mathbf{0}$. Calculating the first iteration

$$ \begin{align*}
    x_{1}^{(1)} &= \frac{1}{5} \left( 14 - 0 + 0 - 0 \right) = 2.8, \\ 
    x_{2}^{(1)} &= \frac{1}{4} \left( 10 - 2.8 + 0 + 0 \right) = 1.8, \\ 
    x_{3}^{(1)} &= \frac{1}{5} \left( -15 + 2.8 + 1.8 - 0 \right) = -2.08, \\ 
    x_{4}^{(1)} &= \frac{1}{3} \left( 3 - 2.8 + 1.8 + (-2.08) \right) = 1.36. 
\end{align*} $$

Calculate the residual

$$ \be

CompletedProcess(args='clip', returncode=0)

:::{solution} indirect-methods-ex-gauss-seidel
:class: dropdown

The Gauss-Seidel method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= \frac{1}{5} \left( 14 - x_{2}^{(k)} + x_{3}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{2}^{(k+1)} &= \frac{1}{4} \left( 10 - x_{1}^{(k+1)} + x_{3}^{(k)} + x_{4}^{(k)} \right), \\ 
    x_{3}^{(k+1)} &= \frac{1}{5} \left( -15 + x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{4}^{(k)} \right), \\ 
    x_{4}^{(k+1)} &= \frac{1}{3} \left( 3 - x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{3}^{(k+1)} \right). 
\end{align*} $$ 

Using starting values of $\mathbf{x} = \mathbf{0}$. Calculating the first iteration

$$ \begin{align*}
    x_{1}^{(1)} &= \frac{1}{5} \left( 14 - 0 + 0 - 0 \right) = 2.8, \\ 
    x_{2}^{(1)} &= \frac{1}{4} \left( 10 - 2.8 + 0 + 0 \right) = 1.8, \\ 
    x_{3}^{(1)} &= \frac{1}{5} \left( -15 + 2.8 + 1.8 - 0 \right) = -2.08, \\ 
    x_{4}^{(1)} &= \frac{1}{3} \left( 3 - 2.8 + 1.8 + (-2.08) \right) = 1.36. 
\end{align*} $$

Calculate the residual

$$ \begin{align*} 
    \mathbf{r}^{(1)} = \mathbf{b} - A \mathbf{x}^{(1)} = 
    \begin{pmatrix} 14 \\ 10 \\ -15 \\ 3 \end{pmatrix} - 
    \begin{pmatrix} 5 & 1 & -1 & 1 \\ 1 & 4 & -1 & -1 \\ -1 & -1 & 5 & 1 \\ 1 & -1 & 1 & 3 \end{pmatrix}
    \begin{pmatrix} 2.8 \\ 1.8 \\ -2.08 \\ 1.36 \end{pmatrix} = 
    \begin{pmatrix} -5.24 \\ -0.72 \\ -1.36 \\ 0 \end{pmatrix}.
\end{align*} $$

Since $\max(| \mathbf{r}^{(1)} |) = 5.24 > 10^{-4}$ we continue iterating. Calculating the second iteration

$$ \begin{align*}
    x_{1}^{(2)} &= \frac{1}{5} \left( 14 - 1.8 + (-2.08) - 1.36 \right) = 1.752, \\ 
    x_{2}^{(2)} &= \frac{1}{4} \left( 10 - 1.752 + (-2.08) + 1.36 \right) = 1.882, \\ 
    x_{3}^{(2)} &= \frac{1}{5} \left( -15 + 1.752 + 1.882 - 1.36 \right) = -2.5452, \\ 
    x_{4}^{(2)} &= \frac{1}{3} \left( 3 - 1.752 + 1.882 + (-2.5452) \right) = 1.89173. 
\end{align*} $$

Calculate the residual

$$ \begin{align*} 
    \mathbf{r}^{(2)} = \mathbf{b} - A \mathbf{x}^{(1)} = 
    \begin{pmatrix} 14 \\ 10 \\ -15 \\ 3 \end{pmatrix} - 
    \begin{pmatrix} 5 & 1 & -1 & 1 \\ 1 & 4 & -1 & -1 \\ -1 & -1 & 5 & 1 \\ 1 & -1 & 1 & 3 \end{pmatrix}
    \begin{pmatrix} 1.752 \\ 1.882 \\ -2.5452 \\ 1.89173 \end{pmatrix} = 
    \begin{pmatrix} -1.07893 \\ 0.0665333 \\ -0.531733 \\ 0 \end{pmatrix}.
\end{align*} $$

Since $\max(| \mathbf{r}^{(2)} |) = 1.07893 > 10^{-4}$ we continue iterating. 
:::

:::{exercise} 
:label: indirect-methods-ex-sor

Repeat {ref}`indirect-methods-ex-jacobi` using the SOR method using the optimum value for the relaxation parameter.
:::


In [188]:
def sor_latex(A, b, omega,tol=1e-4):
    n = len(b)
    x = np.zeros(n)
    ordinal = ["first", "second", "third"]
    string =  f"The SOR method for this system is\n\n"
    string += "$$ \\begin{align*}\n"
    for i in range(n):
        string += f"    x_{{{i+1}}}^{{(k+1)}} &= (1 - \omega) x_{{{i+1}}}^{{(k)}} + \\frac{{\\omega}}{{{A[i,i]}}} \\left( {b[i]}"
        for j in range(n):
            if i == j:
                continue
            if A[i,j] == 1:
                string += f" - x_{{{j+1}}}"
            elif A[i,j] == -1:
                string += f" + x_{{{j+1}}}"
            elif A[i,j] < 0:
                string += f" + {-A[i,j]} x_{{{j+1}}}"
            elif A[i,j] > 0:
                string += f" - {A[i,j]} x_{{{j+1}}}"
            if A[i,j] != 0:
                if j < i:
                    string += f"^{{(k+1)}}"
                else:
                    string += f"^{{(k)}}"
        
        if i == n - 1:
            string += " \\right). \n"
        else:
            string += " \\right), \\\ \n"
            
    string += "\\end{align*} $$ \n\n"
    string += "Using starting values of $\\mathbf{x} = \\mathbf{0}$. "
        
    for k in range(2):

        string += f"Calculating the {ordinal[k]} iteration\n\n"
        string += "$$ \\begin{align*}\n"

        for i in range(n):

            string += f"    x_{{{i+1}}}^{{({k+1})}} &= {1 - omega:0.6} (" + print_number(x[i]) + f") + \\frac{{{omega}}}{{{A[i,i]}}} \\left( {b[i]}"
            sum_ = b[i]
            for j in range(n):

                if i != j:
                    sum_ -= A[i,j] * x[j]
        
                if i == j:
                    continue

                if A[i,j] == 1:
                    if x[j] >= 0:
                        string += " - " + print_number(x[j])
                    else:
                        string += " + (" + print_number(x[j]) + ")"

                elif A[i,j] == -1:
                    if x[j] >= 0:
                        string += " + " + print_number(x[j])
                    else:
                        string += " + (" + print_number(x[j]) + ")"

                elif A[i,j] < 0:
                    string += f" - {-A[i,j]}(" + print_number(x[j]) + ")"
                else:
                    string += f" - {A[i,j]}(" + print_number(x[j]) + ")"

            x[i] = (1 - omega) * x[i] + omega * sum_ / A[i,i]

            string += " \\right) = " + print_number(x[i])
            if i == n - 1:
                string += ". \n"
            else:
                string += ", \\\ \n"
            
        r = b - np.dot(A, x)   
        
        string += "\\end{align*} $$\n\nCalculate the residual\n\n"
        string += "$$ \\begin{align*} \n"
        string += f"    \\mathbf{{r}}^{{({k+1})}} = \\mathbf{{b}} - A \\mathbf{{x}}^{{(1)}} = \n"
        string += "    " + matrix_latex(b) + " - \n    " + matrix_latex(A) + "\n    " + matrix_latex(x)+ " = \n    " + matrix_latex(r) + ".\n"
        string += "\\end{align*} $$\n\n"
        string += f"Since $\\max(| \\mathbf{{r}}^{{({k+1})}} |) = {max(abs(r)):0.6} > 10^{{-4}}$ we continue iterating. "
    
    return string
        
# Define linear system
A = np.array([[5, 1, -1, 1], [1, 4, -1, -1], [-1, -1, 5, 1], [1, -1, 1, 3]])
b = np.array([14, 10, -15, 3])

# Solve linear system
string = ":::{solution} indirect-methods-ex-gauss-seidel\n:class: dropdown\n\n"
string += sor_latex(A, b, 1.0946)

string += "\n:::"

print(string)
# subprocess.run("pbcopy", text=True, input=string)
# subprocess.run("clip", text=True, input=string)

:::{solution} indirect-methods-ex-gauss-seidel
:class: dropdown

The SOR method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= (1 - \omega) x_{1}^{(k)} + \frac{\omega}{5} \left( 14 - x_{2}^{(k)} + x_{3}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{2}^{(k+1)} &= (1 - \omega) x_{2}^{(k)} + \frac{\omega}{4} \left( 10 - x_{1}^{(k+1)} + x_{3}^{(k)} + x_{4}^{(k)} \right), \\ 
    x_{3}^{(k+1)} &= (1 - \omega) x_{3}^{(k)} + \frac{\omega}{5} \left( -15 + x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{4}^{(k)} \right), \\ 
    x_{4}^{(k+1)} &= (1 - \omega) x_{4}^{(k)} + \frac{\omega}{3} \left( 3 - x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{3}^{(k+1)} \right). 
\end{align*} $$ 

Using starting values of $\mathbf{x} = \mathbf{0}$. Calculating the first iteration

$$ \begin{align*}
    x_{1}^{(1)} &= -0.0946 (0) + \frac{1.0946}{5} \left( 14 - 0 + 0 - 0 \right) = 3.06488, \\ 
    x_{2}^{(1)} &= -0.0946 (0) + \frac{1.0946}{4} \left( 10 - 3.06488 + 0 + 0 \right) = 1.8978, \\ 
    x_{3}^{(1)} &= -0.0946 (0) + \fra

CompletedProcess(args='clip', returncode=0)


:::{solution} indirect-methods-ex-sor
:class: dropdown

The iteration matrix for the Jacobi method for this system of linear equations is

$$ \begin{align*}
    T_J &= -D^{-1} (L + U) \\
    &= 
    \begin{pmatrix} 5 & 0 & 0 & 0 \\ 0 & 4 & 0 & 0 \\ 0 & 0 & 5 & 0 \\ 0 & 0 & 0 & 3 \end{pmatrix}^1
    \left(
    \begin{pmatrix} 0 & 0 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ -1 & -1 & 0 & 0 \\ 1 & -1 & 1 & 0 \end{pmatrix} +
    \begin{pmatrix} 0 & 1 & -1 & 1 \\ 0 & 0 & -1 & -1 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 \end{pmatrix}
    \right) \\
    &= 
    \begin{pmatrix} \frac{1}{5} & 0 & 0 & 0 \\ 0 & \frac{1}{4} & 0 & 0 \\ 0 & 0 & \frac{1}{5} & 0 \\ 0 & 0 & 0 & \frac{1}{3} \end{pmatrix}
    \begin{pmatrix} 0 & 1 & -1 & 1 \\ 1 & 0 & -1 & -1 \\ -1 & -1 & 0 & 1 \\ 1 & -1 & 1 & 0 \end{pmatrix}  \\
    &= 
    \begin{pmatrix}
        0 & - \frac{1}{5} & \frac{1}{5} & - \frac{1}{5} \\
        - \frac{1}{4} & 0 & \frac{1}{4} & \frac{1}{4} \\
        \frac{4}{15} & \frac{2}{15} & \frac{1}{15} & - \frac{1}{5} \\
        - \frac{1}{3} & \frac{1}{3} & - \frac{1}{3} & 0
    \end{pmatrix}
\end{align*} $$

and the eigenvalues are $\lambda_1 = -0.5347$, $\lambda_2 = -0.249523$, $\lambda_3 = 0.561913$ and $\lambda_4 =  0.222310$ so $\rho(T_J) = 0.561913$. Using equation {eq}`optimum-relaxation-parameter-equation`

$$ \begin{align*}
    \omega = 1 + \left( \frac{0.561913}{1 + \sqrt{1 - 0.561913^2}} \right)^2 = 1.0946.
\end{align*} $$

The SOR method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= (1 - \omega) x_{1}^{(k)} + \frac{\omega}{5} \left( 14 - x_{2}^{(k)} + x_{3}^{(k)} - x_{4}^{(k)} \right), \\ 
    x_{2}^{(k+1)} &= (1 - \omega) x_{2}^{(k)} + \frac{\omega}{4} \left( 10 - x_{1}^{(k+1)} + x_{3}^{(k)} + x_{4}^{(k)} \right), \\ 
    x_{3}^{(k+1)} &= (1 - \omega) x_{3}^{(k)} + \frac{\omega}{5} \left( -15 + x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{4}^{(k)} \right), \\ 
    x_{4}^{(k+1)} &= (1 - \omega) x_{4}^{(k)} + \frac{\omega}{3} \left( 3 - x_{1}^{(k+1)} + x_{2}^{(k+1)} - x_{3}^{(k+1)} \right). 
\end{align*} $$ 

Using starting values of $\mathbf{x} = \mathbf{0}$. Calculating the first iteration

$$ \begin{align*}
    x_{1}^{(1)} &= -0.0946 (0) + \frac{1.0946}{5} \left( 14 - 0 + 0 - 0 \right) = 3.06488, \\ 
    x_{2}^{(1)} &= -0.0946 (0) + \frac{1.0946}{4} \left( 10 - 3.06488 + 0 + 0 \right) = 1.8978, \\ 
    x_{3}^{(1)} &= -0.0946 (0) + \frac{1.0946}{5} \left( -15 + 3.06488 + 1.8978 - 0 \right) = -2.19737, \\ 
    x_{4}^{(1)} &= -0.0946 (0) + \frac{1.0946}{3} \left( 3 - 3.06488 + 1.8978 + (-2.19737) \right) = 1.47052. 
\end{align*} $$

Calculate the residual

$$ \begin{align*} 
    \mathbf{r}^{(1)} = \mathbf{b} - A \mathbf{x}^{(1)} = 
    \begin{pmatrix} 14 \\ 10 \\ -15 \\ 3 \end{pmatrix} - 
    \begin{pmatrix} 5 & 1 & -1 & 1 \\ 1 & 4 & -1 & -1 \\ -1 & -1 & 5 & 1 \\ 1 & -1 & 1 & 3 \end{pmatrix}
    \begin{pmatrix} 3.06488 \\ 1.8978 \\ -2.19737 \\ 1.47052 \end{pmatrix} = 
    \begin{pmatrix} -6.89008 \\ -1.38292 \\ -0.520986 \\ -0.381265 \end{pmatrix}.
\end{align*} $$

Since $\max(| \mathbf{r}^{(1)} |) = 6.89008 > 10^{-4}$ we continue iterating. Calculating the second iteration

$$ \begin{align*}
    x_{1}^{(2)} &= -0.0946 (3.06488) + \frac{1.0946}{5} \left( 14 - 1.8978 + (-2.19737) - 1.47052 \right) = 1.5565, \\ 
    x_{2}^{(2)} &= -0.0946 (1.8978) + \frac{1.0946}{4} \left( 10 - 1.5565 + (-2.19737) + 1.47052 \right) = 1.93213, \\ 
    x_{3}^{(2)} &= -0.0946 (-2.19737) + \frac{1.0946}{5} \left( -15 + 1.5565 + 1.93213 - 1.47052 \right) = -2.63412, \\ 
    x_{4}^{(2)} &= -0.0946 (1.47052) + \frac{1.0946}{3} \left( 3 - 1.5565 + 1.93213 + (-2.63412) \right) = 2.05365. 
\end{align*} $$

Calculate the residual

$$ \begin{align*} 
    \mathbf{r}^{(2)} = \mathbf{b} - A \mathbf{x}^{(1)} = 
    \begin{pmatrix} 14 \\ 10 \\ -15 \\ 3 \end{pmatrix} - 
    \begin{pmatrix} 5 & 1 & -1 & 1 \\ 1 & 4 & -1 & -1 \\ -1 & -1 & 5 & 1 \\ 1 & -1 & 1 & 3 \end{pmatrix}
    \begin{pmatrix} 1.5565 \\ 1.93213 \\ -2.63412 \\ 2.05365 \end{pmatrix} = 
    \begin{pmatrix} -0.402411 \\ 0.134508 \\ -0.394399 \\ -0.151189 \end{pmatrix}.
\end{align*} $$

Since $\max(| \mathbf{r}^{(2)} |) = 0.402411 > 10^{-4}$ we continue iterating. 
:::

:::{exercise} 
:label: indirect-methods-ex-convergence

Which of the three methods here would you expect to converge to the solution to the system from {ref}`indirect-methods-ex-jacobi` with the fewest iterations? What quantitative evidence do you have to support your conclusion?
:::


In [220]:
def LDU(A):
    L = np.tril(A,-1)
    U = np.triu(A, 1)
    D = A - L - U
    return L, D, U

def jacobi_T(A):
    L, D, U = LDU(A)
    return np.dot(-np.linalg.inv(D), L + U)

def gauss_seidel_T(A):
    L, D, U = LDU(A)
    return np.dot(-np.linalg.inv(L + D), U)

def sor_T(A, omega):
    L, D, U = LDU(A)
    return np.dot(np.linalg.inv(D + omega * L), (1 - omega) * D - omega * U)


omega = 1.0946
TJ = jacobi_T(A)
TGS = gauss_seidel_T(A)
TSOR = sor_T(A, omega)

string = matrix_latex(TSOR)

print(string)

val, _ = np.linalg.eig(TJ)
print(max(np.absolute(val)))

val, _ = np.linalg.eig(TGS)
print(max(np.absolute(val)))

val, _= np.linalg.eig(TSOR)
print(max(np.absolute(val)))

\begin{pmatrix} -0.0946 & -0.21892 & 0.21892 & -0.21892 \\ 0.0258873 & -0.0346925 & 0.213743 & 0.333557 \\ -0.0150426 & -0.0555209 & 0.000118484 & -0.193824 \\ 0.0494503 & 0.0874762 & -0.00193231 & 0.1777 \end{pmatrix}
0.5619126574137866
0.2961489208657579
0.17163529662536403



:::{solution} indirect-methods-ex-convergence
:class: dropdown

The iteration matrices for the Jacobi, Gauss-Seidel and SOR methods (with $\omega = 1.094573$) are

$$ \begin{align*}
    T_J &= \begin{pmatrix} 0 & -0.2 & 0.2 & -0.2 \\ -0.25 & 0 & 0.25 & 0.25 \\ 0.2 & 0.2 & 0 & -0.2 \\ -0.333333 & 0.333333 & -0.333333 & 0 \end{pmatrix}, \\
    T_{GS} &= \begin{pmatrix} 0 & -0.2 & 0.2 & -0.2 \\ 0 & 0.05 & 0.2 & 0.3 \\ 0 & -0.03 & 0.08 & -0.18 \\ 0 & 0.0933333 & -0.0266667 & 0.226667 \end{pmatrix}, \\
    T_{SOR} &= \begin{pmatrix} -0.0946 & -0.21892 & 0.21892 & -0.21892 \\ 0.0258873 & -0.0346925 & 0.213743 & 0.333557 \\ -0.0150426 & -0.0555209 & 0.000118484 & -0.193824 \\ 0.0494503 & 0.0874762 & -0.00193231 & 0.1777 \end{pmatrix}.
\end{align*} $$

and the spectral radii for these matrices are

$$ \begin{align*}
    \rho(T_J) &= 0.561913, \\
    \rho(T_{GS}) &= 0.296149, \\
    \rho(T_{SOR}) &= 0.171610,
\end{align*} $$

so we would expect the SOR method to convergence the fastest. This is supported by the solutions to {ref}`indirect-methods-ex-jacobi`, {ref}`indirect-methods-ex-gauss-seidel` and {ref}`indirect-methods-ex-sor`.
:::


:::{exercise} 
:label: indirect-methods-ex-code

Write a program to calculate the solution to the system of linear equations from {ref}`indirect-methods-ex-jacobi` using the Jacobi, Gauss-Seidel and SOR methods using a convergence tolerance of $tol=10^{-6}$. How many iterations did each of the three methods take to converge to the solution?
:::

::::::{solution} indirect-methods-ex-code
:class: dropdown

:::::{tab-set}
::::{tab-item} Python

```python
import numpy as np

def jacobi(A, b, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    for k in range(maxiter):
        xo = np.copy(x)
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * xo[j]
        
            x[i] = s / A[i,i]
            
        r = b - np.dot(A, x)   
        if max(abs(r)) < tol:
            break
    
    return x, k + 1


def gauss_seidel(A, b, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    for k in range(maxiter):
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * x[j]
        
            x[i] = s / A[i,i]
            
        r = b - np.dot(A, x)   
        if max(abs(r)) < tol:
            break
    
    return x, k + 1


def sor(A, b, omega, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    for k in range(maxiter):
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * x[j]
        
            x[i] = (1 - omega) * x[i] + omega / A[i,i] * s
            
        r = b - np.dot(A, x)   
        if max(abs(r)) < tol:
            break
    
    return x, k + 1


# Define linear system
A = np.array([[5, 1, -1, 1], 
              [1, 4, -1, -1], 
              [-1, -1, 5, 1], 
              [1, -1, 1, 3]])
b = np.array([14, 10, -15, 3])

# Solve linear system
x, iterations = jacobi(A, b)
print(f"Jacobi method:       {iterations:3d} iterations")

x, iterations = gauss_seidel(A, b)
print(f"Gauss-Seidel method: {iterations:3d} iterations")

x, iterations = sor(A, b, 1.094573)
print(f"SOR method:          {iterations:3d} iterations")

print("\nSolution:")
for i in range(len(x)):
    print(f"    x{i+1} = {x[i]:9.6f}")
```
::::

::::{tab-item} MATLAB

```matlab
% Define linear system
A = [5, 1, -1, 1 ; 
     1, 4, -1, -1, ; 
     -1, -1, 5, 1 ; 
     1, -1, 1 3 ];
b = [14, ; 10 ; -15 ; 3];

% Solve linear system
[x, iterations] = jacobi(A, b, 1e-6);
fprintf("Jacobi method:       %3i iterations", iterations)

[x, iterations] = gauss_seidel(A, b, 1e-6);
fprintf("Gauss-Seidel method: %3i iterations", iterations)

[x, iterations] =sor(A, b, 1.094573, 1e-6);
fprintf("Gauss-Seidel method: %3i iterations", iterations)

for i = 1 : length(x)
    fprintf("x%1i = %9.6f\n", i, x(i))
end

% --------------------------------------------------------------
function [x, k] = jacobi(A, b, tol)

n = length(b);
x = zeros(size(b));
maxiter = 100;
for k = 1 : maxiter
    xo = x;
    for i = 1 : n
        s = b(i);
        for j = 1 : n
            if i ~= j
                s = s - A(i,j) * xo(j);
            end
        end
        x(i) = s / A(i,i);
    end
    r = b - A * x;
    if max(abs(r)) < tol
        break
    end
end

end

% --------------------------------------------------------------
function [x, k] = gauss_seidel(A, b, tol)

n = length(b);
x = zeros(size(b));
maxiter = 100;
for k = 1 : maxiter
    for i = 1 : n
        s = b(i);
        for j = 1 : n
            if i ~= j
                s = s - A(i,j) * x(j);
            end
        end
        x(i) = s / A(i,i);
    end
    r = b - A * x;
    if max(abs(r)) < tol
        break
    end
end

end

% --------------------------------------------------------------
function [x, k] = sor(A, b, omega, tol)

n = length(b);
x = zeros(n, 1);
for k = 1 : 100
    for i = 1 : n
        sum_ = 0;
        for j = 1 : n
            if j ~= i
                sum_ = sum_ + A(i,j) * x(j);
            end
        end
        x(i) = (1 - omega) * x(i) + omega * (b(i) - sum_) / A(i,i);
    end
    r = b - A * x;
    if max(abs(r)) < tol
        break
    end
end

end
```

::::
:::::

Output

```
Jacobi method:        27 iterations
Gauss-Seidel method:  14 iterations
SOR method:           10 iterations

Solution:
    x1 =  1.438776
    x2 =  1.979592
    x3 = -2.734694
    x4 =  2.091837
```
::::::

In [1]:
import numpy as np

def jacobi(A, b, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    for k in range(maxiter):
        xo = np.copy(x)
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * xo[j]
        
            x[i] = s / A[i,i]
            
        r = b - np.dot(A, x)   
        if max(abs(r)) < tol:
            break
    
    return x, k + 1


def gauss_seidel(A, b, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    for k in range(maxiter):
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * x[j]
        
            x[i] = s / A[i,i]
            
        r = b - np.dot(A, x)   
        if max(abs(r)) < tol:
            break
    
    return x, k + 1


def sor(A, b, omega, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    for k in range(maxiter):
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * x[j]
        
            x[i] = (1 - omega) * x[i] + omega / A[i,i] * s
            
        r = b - np.dot(A, x)   
        if max(abs(r)) < tol:
            break
    
    return x, k + 1


# Define linear system
A = np.array([[5, 1, -1, 1], 
              [1, 4, -1, -1], 
              [-1, -1, 5, 1], 
              [1, -1, 1, 3]])
b = np.array([14, 10, -15, 3])

# Solve linear system
x, iterations = jacobi(A, b)
print(f"Jacobi method:       {iterations:3d} iterations")

x, iterations = gauss_seidel(A, b)
print(f"Gauss-Seidel method: {iterations:3d} iterations")

x, iterations = sor(A, b, 1.094573)
print(f"SOR method:          {iterations:3d} iterations")

print("\nSolution:")
for i in range(len(x)):
    print(f"    x{i+1} = {x[i]:9.6f}")

Jacobi method:        27 iterations
Gauss-Seidel method:  14 iterations
SOR method:           10 iterations

Solution:
    x1 =  1.438776
    x2 =  1.979592
    x3 = -2.734694
    x4 =  2.091837



:::{exercise}
:label: indirect-methods-ex-convergence-2

A linear system has the following coefficient matrix. What is the largest value that $\alpha$ can be in order for the Jacobi method to be convergent?

$$ \begin{align*}
    A = \begin{pmatrix}
        2 & 1 \\
        \alpha  & 2
    \end{pmatrix}
\end{align*} $$
:::

:::{solution} indirect-methods-ex-convergence-2
:class: dropdown

The iteration matrix for the Jacobi method use to solve this system of linear equations is

$$ \begin{align*}
    T &= -D^{-1}(L + U) 
    = - \begin{pmatrix} 2 & 0 \\ 0 & 2 \end{pmatrix}^{-1}
    \begin{pmatrix} 0 & 1 \\ \alpha & 2 \end{pmatrix} \\
    &= -\begin{pmatrix} \frac{1}{2} & 0 \\ 0 & \frac{1}{2} \end{pmatrix}
    \begin{pmatrix} 0 & 1 \\ \alpha & 2 \end{pmatrix}
    = \begin{pmatrix} 0 & -\frac{1}{2} \\ -\frac{1}{2} \alpha & 0 \end{pmatrix}.
\end{align*} $$

Calculate the eigenvalues of $T_J$

$$ \begin{align*}
    0 &= \begin{vmatrix} -\lambda & -\frac{1}{2} \\ -\frac{1}{2} \alpha & -\lambda \end{vmatrix}
    = \lambda^2 - \frac{1}{4} \alpha, \\
    \therefore \lambda &= \frac{1}{2} \sqrt{\alpha}
\end{align*} $$

For a method to converge $\rho(T) < 1$ so $\frac{1}{2} \sqrt{\alpha} < 1$ therefore $\alpha < 4$.
:::